In [1]:
%load_ext Cython
from numpy import linalg
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
%%cython
#####################################CLASE PARTÍCULAS###############################
import numpy as np

class Particula:#En esta clase definimos los métodos fundamentales que debe seguir cada partícula

    def __init__(self, (float,float) posicion, (float,float) velocidad, float masa, float radio): 
       """comenzamos definiendo el método que debe 
        inicializar la clase, en este definimos los atributos básicos de la clase partícula
        las entradas masa y radio son de tipo float y las entradas de posición son tuplas conformadas por floats"""  
       self.radio=radio  
       self.masa=masa 
       
       self.posicion=np.array(posicion)   #Estas variables son arrays de numpy, y las trataremos como vectores
       self.velocidad=np.array(velocidad)
       self.velocidad_mag=np.linalg.norm(self.velocidad)#Este atributo no es otra cosa que la magnitud del vector velocidad

     
  ##########################################################################
    def paso_dt(self,float dt):   #Falta definir el valor que va a tomar dt  
       """Este método lo que hace es avanzar en el tiempo, cambia la posición de la partícula"""
      
       self.posicion=self.posicion + self.velocidad*dt  
      
 ###############################################################################################
    def ver_colision_pp(self,otra_p):
       """Este método verifica si se dio luegar a una colisión entre dos partículas, las entradas
      son dos partículas, se definen sus radios y posiciones y se plantea una condición que indica 
      si las partículas chocaron"""
       cdef float r1=self.radio
       cdef float r2=otra_p.radio
       p1=self.posicion
       p2=otra_p.posicion
       cdef float sep=np.linalg.norm(p1-p2)#Norma del vector separación de ambas partículas.
       if sep-(r1+r2)*1.1<=0:#Si la separación es menor o igual a la suma de sus radios, entonces las partículas están en contacto y por lo tanto chocaron.
         return True
       else:
         return False
######################################################################################################
    def ver_colision_esquina(self, float Lx, float Ly):
       "Revisa si hay colisión con una esquina, Lx y Ly son las dimensiones horizontal y vertical de la caja respectivamente"
       extremo_izquierdo = self.posicion[0] - self.radio #da la posición del extremo izquido de la partícula
       extremo_inferior = self.posicion[1] - self.radio  #da la posición del extremo inferior de la partícula
       extremo_derecho  = self.posicion[0] + self.radio  #da la posición del extremo derech0 de la partícula
       extremo_superior = self.posicion[1] + self.radio  #da la posición del extremo superior de la partícula
       #Ahora construimos las variables donde se va a almacenar el hecho de chocar contra una esquina o no
       cdef bint choque_00 = extremo_izquierdo >0 and extremo_inferior> 0   #Esto corresponde a no chocar con la esquina (0,0)
       cdef bint choque_Lx0 = extremo_derecho <Lx and extremo_inferior > 0 #Esto corresponde a no chocar con la esquina (Lx,0)
       cdef bint choque_LxLy =  extremo_derecho <Lx and extremo_superior <Ly  #Esto corresponde a no chocar con la esquina (Lx,Ly)
       cdef bint choque_0Ly = extremo_izquierdo > 0 and extremo_superior <Ly  #Esto corresponde a chocar con la esquina (0,Ly)
       if not choque_00 or not choque_Lx0 or not choque_LxLy or not choque_0Ly:
         return  True 
       else:
         return False
       
        
#####################################################################################################
    def ver_colision_muro(self, float Lx, float Ly):
        """Este método revisa si la partícula ha chocado contra un muro."""
        """La condición ve que si la posición en "x" es diferente 0 o la longitud horizontal (Lx) de la caja y adicionalmente
     #  si la posición en "y" es diferente de 0 o la longitud vertical (Ly) entonces la partícula no está chocando contra un muro"""
        cdef bint choque_pared_derecha =  self.posicion[0] + self.radio <Lx   
        cdef bint choque_pared_izquierda = self.posicion[0] - self.radio>0
        #las condiciones corresponden a no chocar con esas paredes
        cdef bint choque_pared_superior = self.posicion[1] + self.radio <Ly 
        cdef bint choque_pared_inferior = self.posicion[1] - self.radio >0
        if choque_pared_derecha and choque_pared_izquierda and choque_pared_superior and choque_pared_inferior :
          return False
        else:
          return True 
        
######################################################################################################
    def resolver_colision_particula(self,otra_p):
      """Método que actualiza las velocidades de dos partículas después de chocar
       Definimos las magnitudes a usar para resolver el choque"""
      M1=self.masa
      M2=otra_p.masa
      p1=self.posicion
      p2=otra_p.posicion
      V1=self.velocidad
      V2=otra_p.velocidad
      cdef bint choque=self.ver_colision_pp
      if choque:
        self.velocidad= self.velocidad-((2*M2)/(M1+M2))*(np.dot(p1-p2,V1-V2)/(np.linalg.norm(p1-p2)**2))*(p1-p2)#Se resuleven los choques tal cual el modelo bidimensional que se tiene
        otra_p.velocidad= otra_p.velocidad-((2*M1)/(M1+M2))*(np.dot(p2-p1,V2-V1)/(np.linalg.norm(p2-p1)**2))*(p2-p1)
  
###############################################################################################################
    def resolver_colision_muro(self,float Lx,float Ly): 
        """#Método que actualiza la velocidad después de que una partícula choca con un muro. Recibe la
        partícula y las dimensiones de la caja"""
        cdef bint choque_pared_derecha =  self.posicion[0] + self.radio <Lx   
        cdef bint choque_pared_izquierda = self.posicion[0] - self.radio>0
        #las condiciones corresponden a no chocar con esas paredes
        cdef bint choque_pared_superior = self.posicion[1] + self.radio <Ly 
        cdef bint choque_pared_inferior = self.posicion[1] - self.radio >0
        #Ahora miramos cual fue la pared con que se chocó e invertimos la coordenada teniendo en cuenta eso
        if not choque_pared_derecha or not choque_pared_izquierda:
            self.velocidad[0] = -1* self.velocidad[0]
        elif not choque_pared_superior or not choque_pared_inferior:
            self.velocidad[1] = -1* self.velocidad[1]
       
    def resolver_colision_esquina(self):
        """Método que actualiza la velocidad después de que una partícula choca con una esquina"""
        self.velocidad = -1* self.velocidad #se invierte todo el vector
            
        

In [3]:
#Prototipo de la función final, recibe las dimensiones de la caja, el rango de velocidades inciales, el número
# de partículas, el radio y la masa
#Si algo podemos establecer por defecto que las masas y los radios sean los mismos por defecto o cómo quieran


######Función que generá las partículas con datos aleatorios
def funcion_simuladora(lx,ly,v1,v2,m,r,n,):
#Por el momento se crea un arreglo vacío de enteros para las posiciones iniciales
  posiciones=np.zeros((n,2))
  
  for i in range(n):
    #Se llena el arreglo con números aleatorios dentro de la caja pero evitando que aparezcan en las esquinas
    posiciones[i][0]=np.random.uniform(10,lx-10)
    posiciones[i][1]=np.random.uniform(10,ly-10)
  #Se convierte el arreglo en una lista
  posiciones_lista=list(posiciones)
  #Las velocidades se crean normalmente con velocidades aleatorias dentro del rango establecido, el intervalo (v1,v2)
  velocidades_lista=list(np.random.uniform(v1,v2,(n,2)))
  #Ahora creamos una lista vacía donde irán objetos de la clase partícula
  lista_de_particulas=[]
  #Llenamos la lista con partículas que tendrán posiciones provenientes de posiciones_lista y velocidades_lista. 
  #Las masas y radios son los determinados al activar la función.
  for j in range(n):
    Pn=Particula(tuple(posiciones_lista[j]),tuple(velocidades_lista[j]),m,r)
    lista_de_particulas.append(Pn)
  
  
  #Ahora queremeos hacer que si las partículas se crean superpuestas de alguna manera, eliminamos una y creamos otra nueva
  #luego añadimos la nueva partícula a la lista de partículas

  for i in range(n):
   for p1 in lista_de_particulas:
    for p2 in lista_de_particulas:
     if p1.ver_colision_pp(p2):
       p_nueva=Particula(tuple([(random.uniform(10,lx-10),random.uniform(0,ly-10)) for x in range(1)][0]),
       tuple([(random.uniform(v1,v2),random.uniform(v1,v2)) for x in range(1)][0]), m,r)
                                                      
       lista_de_particulas.pop(i)
       lista_de_particulas.append(p_nueva)
  #Nos retorna la lista con las partículas creadas, acá le puse que me devolviera solo la primera para demostrar funcionalidad

  return lista_de_particulas


In [4]:
#Funciones que generan las listas que después se van a llenar con los datos

def listas_para_llenar_posiciones(n): ###Recibe el número de partículas y crea una lista con sublistas. 
    lista_principal = []   ###Cada partícula tiene una lista que ocupa una posicion en lista_principal
    for j in range(n):     ###y a su vez estas listas tienen dos listas para almacenar la información 
        lista_principal.append([[],[]]) ###de la posición en x y y de esa partícula.
    return lista_principal

def listas_para_llenar_velocidad(nt): ###Recibe la cantidad de pasos y genera una lista con listas para cada paso.
    lista_principal = []              ###La idea es que cada objeto de la lista sea una lista con TODAS las 
    for j in range(nt):               ###velocidades de ese paso.
        lista_principal.append([])
    return lista_principal

In [5]:
#Pruebas de la simulación
#definimos las condiciones
v1 = 50  #mínimo, intervalo de velocidades
v2 = 100 #máximo, intervalo de velocidades
dt = 0.01 #tamaño del paso
Lx = 200 #Tamaño de la caja        
Ly = 200
nt = 1000 #número de pasos
n = 150 #número de partículas
m = 1 #masa 
r = 2 #radio
par = funcion_simuladora(Lx,Ly,v1,v2,m,r,n) #Lista con las partículas
lista_posiciones = listas_para_llenar_posiciones(n)
lista_magnitudes_velocidad = listas_para_llenar_velocidad(nt) #Esta lista tiene los datos de la magnitud de las
                                                              #velocidades en cada paso


for j in range(nt): #Empezamos por definir un loop que corresponde con cada paso 
    for p in par:  #iteramos sobre la lista de partículas 
        lista_magnitudes_velocidad[j].append(p.velocidad_mag) #añadimos la magnitud de cada partícula a la lista
                                                              #que lleva las magnitudes en cada paso
   
    for k in range(n):  #Añadimos los datos correspondientes a la posición de cada partícula                                    
        lista_posiciones[k][0].append(par[k].posicion[0])  #Se añade la posición en x de cada partícula
        lista_posiciones[k][1].append(par[k].posicion[1])  #Se añade la posicion en y de cada partícula
    
    for p in par:  #Revisamos las colisiones entre partículas        
        for i in range(par.index(p)+1,n): #Tomamos p y la revisamos con todas las partículas que tiene adelante
                                          #en la lista, así podemos reducir la cantidad de operaciones y no se
                                          #revisa dos veces cada pareja
            if i == n-1:                            
                break  #Esto es para evitar que la última partícula se evalue consigo misma
            elif p.ver_colision_pp(par[i]):  #se detecta si hay colisión o no con la partícula que se está revisando
                p.resolver_colision_particula(par[i]) #se resuelve la colisión
    
    for p in par:  #Revisamos las condiciones de choque con la caja
        if p.ver_colision_muro(Lx,Ly): #Revisamos las condiciones de choque con los muros
            p.resolver_colision_muro(Lx,Ly)
        elif p.ver_colision_esquina(Lx,Ly): #Revisamos las condiciones de choque con las esquinas
            p.resolver_colision_esquina()
        p.paso_dt(dt)



In [7]:
#############FUNCIÓN QUE ESCRIBE LOS DATOS DE LAS MAGNITUDES DE VELOCIDAD EN .csv###################
#Los datos de las velocidades en .csv
import csv 
with open('datos_magnitud_velocidades','w',newline = '') as file:
    writer = csv.writer(file)
    for j in range(nt):
        writer.writerow(lista_magnitudes_velocidad[j])